# Maximum Number of K-Divisible Components


There is an undirected tree with n nodes labeled from 0 to n - 1. You are given the integer n and a 2D integer array edges of length n - 1, where edges\[i] = \[ai, bi] indicates that there is an edge between nodes ai and bi in the tree.

You are also given a 0-indexed integer array values of length n, where values\[i] is the value associated with the ith node, and an integer k.

A valid split of the tree is obtained by removing any set of edges, possibly empty, from the tree such that the resulting components all have values that are divisible by k, where the value of a connected component is the sum of the values of its nodes.

Return the maximum number of components in any valid split.


The solution consist of post-order DFS.

The algorithm ->


*   Start from node 0
*   For each subtree, compute the total sun
*   If the subtree's sum is divided by k and it's not the root; cut off that edge
*   The return value is the remaining sum that continues up the tree(0 if it was cut)
*   At the end, add 1 to the answer to include root component

In [3]:
%%writefile 2872.cpp
class Solution {
public:
    int ans = 0;
    int dfs(int node, int parent, vector<vector<int>>& tree, vector<int> &values, int k){
        long long total = values[node];
        for(int child: tree[node]){
            if(child != parent){
                long long subtotal = dfs(child, node, tree, values, k);
                total += subtotal;
            }
        }

        if(total%k == 0 & node != 0){
            ans++;
            return 0;
        }

        return total;
    }
    int maxKDivisibleComponents(int n, vector<vector<int>>& edges, vector<int>& values, int k) {
        vector<vector<int>> tree(n);
        for(auto& edge: edges){
            int u = edge[0], v = edge[1];
            tree[u].push_back(v);
            tree[v].push_back(u);
        }
        long long totalSum = 0;
        for(int value: values){
            totalSum += value;
        }
        if (totalSum % k != 0) return 0;

        dfs(0, -1, tree, values, k);
        return ans + 1;
    }
};

Writing 2872.cpp


Here note that sum was taken to check if it was feasible to build any valid components from here.

Then I dropped calculating totalSum and return ans + 1. Maybe in the testcases there are no trees which we can't make any components. I used,

```
assert(totalSum % k == 0);  // This will crash if false
```

None of them failed. And it performs better than the code without having that part.

I asked ChatGPT!!!

It says calculating the total sums upfront **before random access later can speed up total performance due to prefetching/caching**. And also modern cpus after few runs learn to skip this part, if it never evaluates false. This is because of the the **CPU predicts the likely outcome using recent history**.

This is the leetcode implementation. Changes fro mine to them:


```
 auto child: tree[node]
```

Using componentCount to calculate the number of components without getting the totalSum.

In [6]:
%%writefile 2872_leetcode.cpp
class Solution {
public:
    int maxKDivisibleComponents(int n, vector<vector<int>> &edges,
                                vector<int> &values, int k) {
        // Step 1: Create adjacency list from edges
        vector<int> adjList[n];
        for (auto edge : edges) {
            int node1 = edge[0];
            int node2 = edge[1];
            adjList[node1].push_back(node2);
            adjList[node2].push_back(node1);
        }
        // Step 2: Initialize component count
        int componentCount = 0;

        // Step 3: Start DFS traversal from node 0
        dfs(0, -1, adjList, values, k, componentCount);

        // Step 4: Return the total number of components
        return componentCount;
    }

private:
    int dfs(int currentNode, int parentNode, vector<int> adjList[],
            vector<int> &nodeValues, int k, int &componentCount) {
        // Step 1: Initialize sum for the current subtree
        int sum = 0;

        // Step 2: Traverse all neighbors
        for (auto neighborNode : adjList[currentNode]) {
            if (neighborNode != parentNode) {
                // Recursive call to process the subtree rooted at the neighbor
                sum += dfs(neighborNode, currentNode, adjList, nodeValues, k,
                           componentCount);
                sum %= k;  // Ensure the sum stays within bounds
            }
        }

        // Step 3: Add the value of the current node to the sum
        sum += nodeValues[currentNode];

        // Step 4: Check if the sum is divisible by k
        sum %= k;
        if (sum == 0) componentCount++;

        // Step 5: Return the computed sum for the current subtree
        return sum;
    }
};

Overwriting 2872_leetcode.cpp
